ADZUNA Kaggle Project

- Review the nltk NLP library in Python
- Tokenize sentences
- Remove stopwords + Others

NLP in Python: http://www.nltk.org/book/ch01.html

Stem words: http://www.nltk.org/book/ch03.html  (how to remove plural endings and stemming)

Most popular phrases: http://www.markhneedham.com/blog/2015/01/19/pythonnltk-finding-the-most-common-phrases-in-how-i-met-your-mother/

Useful Python Tools: https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/


In [124]:
import pandas as pd                  # import necessary libraries
import nltk
from nltk.corpus import stopwords
from nltk import util
from nltk import ngrams

In [2]:
#Import each of the csv files you downloaded from the SSCC into a pandas df
adzuna = pd.read_csv('~/Dropbox/NU/ADVANCED_MODELING/ADZUNA/datasets/Train_rev1.csv') 

In [3]:
adzuna.head(2)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk


In [11]:
des = adzuna.iloc[0]['FullDescription']

In [12]:
des

'Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K'

In [56]:
desToken = nltk.word_tokenize(des)
desToken[:5]

['Engineering', 'Systems', 'Analyst', 'Dorking', 'Surrey']

In [16]:
desFreq = nltk.FreqDist(desToken)
desFreq.most_common(10)

[(',', 6),
 ('Analyst', 3),
 ('Dorking', 3),
 ('Engineering', 3),
 ('Surrey', 3),
 ('Systems', 3),
 ('****K', 2),
 ('Modelling', 2),
 ('Salary', 2),
 ('client', 2)]

In [15]:
nltk.FreqDist(desToken)

Counter({'****K': 2,
         ',': 6,
         'Analysis': 1,
         'Analyst': 3,
         'Dorking': 3,
         'Engineering': 3,
         'Keywords': 1,
         'MISER': 1,
         'Mathematical': 1,
         'Modelling': 2,
         'Optimisation': 1,
         'Our': 1,
         'PIONEEER': 1,
         'Risk': 1,
         'Salary': 2,
         'Surrey': 3,
         'System': 1,
         'Systems': 3,
         'and': 1,
         'are': 1,
         'client': 2,
         'development': 1,
         'for': 1,
         'in': 1,
         'is': 1,
         'located': 1,
         'looking': 1,
         'our': 1,
         'provides': 1,
         'software': 1,
         'specialist': 1})

In [39]:
stops = set(stopwords.words("english"))
stops.update([',','****K'])
filtered_words = [word for word in desToken if word not in stops]  # stopwords.words('english')]
filtered_words

['Engineering',
 'Systems',
 'Analyst',
 'Dorking',
 'Surrey',
 'Salary',
 'Our',
 'client',
 'located',
 'Dorking',
 'Surrey',
 'looking',
 'Engineering',
 'Systems',
 'Analyst',
 'client',
 'provides',
 'specialist',
 'software',
 'development',
 'Keywords',
 'Mathematical',
 'Modelling',
 'Risk',
 'Analysis',
 'System',
 'Modelling',
 'Optimisation',
 'MISER',
 'PIONEEER',
 'Engineering',
 'Systems',
 'Analyst',
 'Dorking',
 'Surrey',
 'Salary']

In [40]:
filtered_words_lower = [x.lower() for x in filtered_words]
filtered_words_lower

['engineering',
 'systems',
 'analyst',
 'dorking',
 'surrey',
 'salary',
 'our',
 'client',
 'located',
 'dorking',
 'surrey',
 'looking',
 'engineering',
 'systems',
 'analyst',
 'client',
 'provides',
 'specialist',
 'software',
 'development',
 'keywords',
 'mathematical',
 'modelling',
 'risk',
 'analysis',
 'system',
 'modelling',
 'optimisation',
 'miser',
 'pioneeer',
 'engineering',
 'systems',
 'analyst',
 'dorking',
 'surrey',
 'salary']

In [41]:
nltk.FreqDist(filtered_words_lower)

Counter({'analysis': 1,
         'analyst': 3,
         'client': 2,
         'development': 1,
         'dorking': 3,
         'engineering': 3,
         'keywords': 1,
         'located': 1,
         'looking': 1,
         'mathematical': 1,
         'miser': 1,
         'modelling': 2,
         'optimisation': 1,
         'our': 1,
         'pioneeer': 1,
         'provides': 1,
         'risk': 1,
         'salary': 2,
         'software': 1,
         'specialist': 1,
         'surrey': 3,
         'system': 1,
         'systems': 3})

In [42]:
desFreq = nltk.FreqDist(filtered_words_lower)
desFreq.most_common(10)

[('engineering', 3),
 ('systems', 3),
 ('surrey', 3),
 ('dorking', 3),
 ('analyst', 3),
 ('modelling', 2),
 ('salary', 2),
 ('client', 2),
 ('risk', 1),
 ('software', 1)]

In [82]:
from nltk import ngrams
#sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 2
#sixgrams = ngrams(sentence.split(), n)
ngrams = ngrams(filtered_words_lower, n)
ngrams

ngram_sentence = []
for grams in ngrams:
 ngram_sentence.append(grams)

ngram_sentence

[('engineering', 'systems'),
 ('systems', 'analyst'),
 ('analyst', 'dorking'),
 ('dorking', 'surrey'),
 ('surrey', 'salary'),
 ('salary', 'our'),
 ('our', 'client'),
 ('client', 'located'),
 ('located', 'dorking'),
 ('dorking', 'surrey'),
 ('surrey', 'looking'),
 ('looking', 'engineering'),
 ('engineering', 'systems'),
 ('systems', 'analyst'),
 ('analyst', 'client'),
 ('client', 'provides'),
 ('provides', 'specialist'),
 ('specialist', 'software'),
 ('software', 'development'),
 ('development', 'keywords'),
 ('keywords', 'mathematical'),
 ('mathematical', 'modelling'),
 ('modelling', 'risk'),
 ('risk', 'analysis'),
 ('analysis', 'system'),
 ('system', 'modelling'),
 ('modelling', 'optimisation'),
 ('optimisation', 'miser'),
 ('miser', 'pioneeer'),
 ('pioneeer', 'engineering'),
 ('engineering', 'systems'),
 ('systems', 'analyst'),
 ('analyst', 'dorking'),
 ('dorking', 'surrey'),
 ('surrey', 'salary')]

In [85]:
desFreq = nltk.FreqDist(ngram_sentence)
desFreq.most_common(3)                   # most common 2 letter words

[(('engineering', 'systems'), 3),
 (('dorking', 'surrey'), 3),
 (('systems', 'analyst'), 3)]

Next we need to loop through each row in the adzuna data frame so that we add (initially) the most common word in a new column of the adzuna data frame. Let's first create a copy of the data frame. 

In [86]:
df1 = adzuna

In [110]:
def get_most_common_word(des):
    token = nltk.word_tokenize(des)                                    # split sentence into words
    stops = set(stopwords.words("english"))                            # create set of stop words
    stops.update([',','****K'])                                        # add additional words to stop words 'set'
    filtered_words = [word for word in token if word not in stops]     # filter out ALL stop words
    filtered_words_lower = [x.lower() for x in filtered_words]         # convert filtered list to lower case
    filtered_words_lower_freq = nltk.FreqDist(filtered_words_lower)    # get frequency of each word in des text
    return(filtered_words_lower_freq.most_common(1))

In [137]:
df1_test = df1['FullDescription'].head()
df1_test.to_frame()

,FullDescription
0,Engineering Systems Analyst Dorking Surrey Sal...
1,Stress Engineer Glasgow Salary **** to **** We...
2,Mathematical Modeller / Simulation Analyst / O...
3,Engineering Systems Analyst / Mathematical Mod...
4,"Pioneer, Miser Engineering Systems Analyst Do..."


In [145]:
# NEXT 
Need to figure out how to apply the get_most_common_word function to each row in data frame. 
df1_test.apply(get_most_common_word)

TypeError: expected string or bytes-like object

In [129]:
for index, row in df1_test.iterrows():
    print(index)
    # do some logic here
#df1.apply(get_most_common_word(x), axis=1)

AttributeError: 'Series' object has no attribute 'iterrows'